In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "cavallo": "horse", "ragno": "spider"}
label_names = list(translate.keys())
label_names.sort()

In [ ]:
# Hyperparameters
learning_rate = 0.001
batch_size = 128

The structure of the directory is simple. Hence, we can use built-in Keras preprocessing. 

In [ ]:
from tensorflow import keras
dir_name = "../input/animals10/raw-img"
train_data = keras.preprocessing.image_dataset_from_directory(dir_name, labels='inferred', label_mode='categorical',
                                                           shuffle=True,seed=1200, batch_size=batch_size, subset="training", validation_split=0.2) 
val_data = keras.preprocessing.image_dataset_from_directory(dir_name, labels='inferred', label_mode='categorical',
                                                           shuffle=False,seed=None, batch_size=batch_size, subset="validation", validation_split=0.2)

In [ ]:
Input_shape = (256, 256, 3)
# Callbacks
reduce_lr_on_plateau = keras.callbacks.ReduceLROnPlateau(min_lr = 1e-8, verbose=1)
model_checkpoint = keras.callbacks.ModelCheckpoint("Inception_model", save_best_only=True)

In [ ]:
def model(input_shape):
    image = keras.layers.Input(input_shape)
    inception_layer = keras.applications.InceptionV3(include_top=False, input_shape = input_shape)
    inception_out = inception_layer(image)
    X = keras.layers.Flatten()(inception_out)
    #X = keras.layers.Dropout(0.2)(X)
    X = keras.layers.Dense(128, activation='relu')(X)
    #X = keras.layers.Dropout(0.2)(X)
    out = keras.layers.Dense(10, activation='softmax')(X)
    
    
    model = keras.Model(inputs = image, outputs = out, name = "Inception_Classifer")
    return model

In [ ]:
inception_classifier = model(Input_shape)
inception_classifier.summary()

In [ ]:
adam_opt = keras.optimizers.Adam(learning_rate=learning_rate)
inception_classifier.compile(optimizer = adam_opt, loss = "categorical_crossentropy", 
                             metrics = ['categorical_accuracy'])

In [ ]:
inception_classifier.fit(train_data, epochs=10, validation_data=val_data, 
                         callbacks=[reduce_lr_on_plateau, model_checkpoint], use_multiprocessing=True, workers=2)